In [ ]:
# Predicción de Abandono de Clientes Bancarios
# Autor: Jonathan Ibáñez
# Fecha: Septiembre 2025
# 
# Modelo de Machine Learning para predecir abandono de clientes bancarios
# Modelo final alcanza AUC: 0.88

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

# Configuración
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)

# =============================================================================
# CARGA Y EXPLORACIÓN DE DATOS
# =============================================================================

# Cargar dataset
df = pd.read_csv('../data/bank_customer_churn.csv')
print(f"Dimensiones del dataset: {df.shape}")
print(f"Uso de memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Información básica
print("\nResumen del dataset:")
print(f"Total de clientes: {df.shape[0]:,}")
print(f"Características disponibles: {df.shape[1]}")
print(f"Variables numéricas: {df.select_dtypes(include=[np.number]).shape[1]}")
print(f"Variables categóricas: {df.select_dtypes(include=['object']).shape[1]}")

# Análisis de variable objetivo
distribucion_abandono = df['Exited'].value_counts()
tasa_abandono = df['Exited'].value_counts(normalize=True)

print(f"\nAnálisis de abandono:")
print(f"Clientes retenidos: {distribucion_abandono[0]:,} ({tasa_abandono[0]:.1%})")
print(f"Clientes que abandonaron: {distribucion_abandono[1]:,} ({tasa_abandono[1]:.1%})")

# Visualizar distribución objetivo
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

distribucion_abandono.plot(kind='bar', ax=ax1, color=['green', 'red'], alpha=0.7)
ax1.set_title('Distribución de Abandono de Clientes')
ax1.set_xlabel('Estado del Cliente')
ax1.set_ylabel('Cantidad')
ax1.set_xticklabels(['Retenidos', 'Abandonaron'], rotation=0)

ax2.pie(distribucion_abandono.values, labels=['Retenidos', 'Abandonaron'], 
        autopct='%1.1f%%', colors=['green', 'red'])
ax2.set_title('Distribución de Tasa de Abandono')

plt.tight_layout()
plt.show()

# =============================================================================
# PREPROCESAMIENTO DE DATOS
# =============================================================================

def preparar_caracteristicas(df):
    """
    Preparar características para modelos de machine learning
    """
    df_procesado = df.copy()
    
    # Identificar columnas categóricas y numéricas
    columnas_categoricas = df_procesado.select_dtypes(include=['object']).columns.tolist()
    columnas_numericas = df_procesado.select_dtypes(include=[np.number]).columns.tolist()
    
    # Remover columnas innecesarias
    columnas_excluir = ['RowNumber', 'CustomerId', 'Surname', 'Exited']
    columnas_categoricas = [col for col in columnas_categoricas if col not in columnas_excluir]
    columnas_numericas = [col for col in columnas_numericas if col not in columnas_excluir]
    
    print(f"Características categóricas: {len(columnas_categoricas)}")
    print(f"Características numéricas: {len(columnas_numericas)}")
    
    # Codificar variables categóricas
    codificadores = {}
    for col in columnas_categoricas:
        le = LabelEncoder()
        df_procesado[col] = le.fit_transform(df_procesado[col])
        codificadores[col] = le
        print(f"Codificado {col}: {len(le.classes_)} valores únicos")
    
    # Selección final de características
    columnas_caracteristicas = columnas_categoricas + columnas_numericas
    X = df_procesado[columnas_caracteristicas]
    y = df_procesado['Exited']
    
    print(f"\nDataset final:")
    print(f"Características (X): {X.shape}")
    print(f"Variable objetivo (y): {y.shape}")
    print(f"Distribución de clases: {np.bincount(y)}")
    
    return X, y, columnas_caracteristicas, codificadores

# Procesar características
X, y, nombres_caracteristicas, codificadores = preparar_caracteristicas(df)

# División entrenamiento-prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\nDivisión de datos completada:")
print(f"Conjunto entrenamiento: {X_train.shape[0]} muestras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Conjunto prueba: {X_test.shape[0]} muestras ({X_test.shape[0]/len(X)*100:.1f}%)")

# Escalado de características para algoritmos que lo requieren
escalador = StandardScaler()
X_train_escalado = escalador.fit_transform(X_train)
X_test_escalado = escalador.transform(X_test)

print(f"Escalado de características completado")
print(f"Características escaladas - Media: {X_train_escalado.mean():.2f}, Std: {X_train_escalado.std():.2f}")

# =============================================================================
# ENTRENAMIENTO Y COMPARACIÓN DE MODELOS
# =============================================================================

# Inicializar modelos
modelos = {
    'SVM': SVC(C=1.0, kernel='rbf', probability=True, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'LightGBM': LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, 
                               random_state=42, verbose=-1),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

# Entrenar y evaluar modelos
resultados = {}
objetos_modelo = {}

print("Entrenando modelos de machine learning:")
print("=" * 50)

for nombre, modelo in modelos.items():
    print(f"\nEntrenando {nombre}...")
    
    # Usar datos escalados para SVM y KNN
    if nombre in ['SVM', 'KNN']:
        modelo.fit(X_train_escalado, y_train)
        y_prob = modelo.predict_proba(X_test_escalado)[:, 1]
        y_pred = modelo.predict(X_test_escalado)
    else:
        modelo.fit(X_train, y_train)
        y_prob = modelo.predict_proba(X_test)[:, 1]
        y_pred = modelo.predict(X_test)
    
    # Calcular métricas
    puntaje_auc = roc_auc_score(y_test, y_prob)
    precision = (y_pred == y_test).mean()
    
    resultados[nombre] = puntaje_auc
    objetos_modelo[nombre] = modelo
    
    print(f"Puntaje AUC: {puntaje_auc:.4f}")
    print(f"Precisión: {precision:.4f}")

# Resumen de resultados
print(f"\n" + "="*50)
print("RESULTADOS DE COMPARACIÓN DE MODELOS")
print("="*50)

resultados_ordenados = dict(sorted(resultados.items(), key=lambda x: x[1], reverse=True))
print("Ranking por Puntaje AUC:")
for i, (nombre_modelo, puntaje_auc) in enumerate(resultados_ordenados.items(), 1):
    print(f"{i}. {nombre_modelo:<15} AUC: {puntaje_auc:.4f}")

mejor_modelo_nombre = max(resultados, key=resultados.get)
print(f"\nMejor modelo: {mejor_modelo_nombre} (AUC: {resultados[mejor_modelo_nombre]:.4f})")

# =============================================================================
# OPTIMIZACIÓN DE HIPERPARÁMETROS
# =============================================================================

print(f"\nOptimizando hiperparámetros de {mejor_modelo_nombre}...")

# Definir grilla de parámetros para LightGBM
grilla_parametros = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [6, 9],
    'num_leaves': [15, 31]
}

# Búsqueda en grilla
busqueda_grilla = GridSearchCV(
    LGBMClassifier(random_state=42, verbose=-1),
    grilla_parametros,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

busqueda_grilla.fit(X_train, y_train)

print(f"\nOptimización completada!")
print(f"Mejores parámetros: {busqueda_grilla.best_params_}")
print(f"Mejor puntaje CV: {busqueda_grilla.best_score_:.4f}")

# Evaluar modelo optimizado
modelo_optimizado = busqueda_grilla.best_estimator_
y_prob_optimizado = modelo_optimizado.predict_proba(X_test)[:, 1]
y_pred_optimizado = modelo_optimizado.predict(X_test)
auc_optimizado = roc_auc_score(y_test, y_prob_optimizado)

print(f"AUC del modelo optimizado en conjunto de prueba: {auc_optimizado:.4f}")
print(f"Mejora: {auc_optimizado - resultados[mejor_modelo_nombre]:.4f}")

# =============================================================================
# EVALUACIÓN E INTERPRETACIÓN DEL MODELO
# =============================================================================

# Evaluación detallada de métricas
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred_optimizado)
recall = recall_score(y_test, y_pred_optimizado)
f1 = f1_score(y_test, y_pred_optimizado)
precision_final = (y_pred_optimizado == y_test).mean()

print(f"\nRendimiento del Modelo Final:")
print(f"Puntaje AUC: {auc_optimizado:.4f}")
print(f"Precisión: {precision_final:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_optimizado)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Retenidos', 'Abandonaron'], 
            yticklabels=['Retenidos', 'Abandonaron'])
plt.title('Matriz de Confusión - Modelo LightGBM Optimizado')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()

# Análisis de importancia de características
df_importancia = pd.DataFrame({
    'caracteristica': nombres_caracteristicas,
    'importancia': modelo_optimizado.feature_importances_
}).sort_values('importancia', ascending=False)

print(f"\nTop 10 Características Más Importantes:")
for i, row in df_importancia.head(10).iterrows():
    print(f"{i+1:2d}. {row['caracteristica']:<20} {row['importancia']:.4f}")

# Visualizar importancia de características
plt.figure(figsize=(10, 8))
top_caracteristicas = df_importancia.head(10)
plt.barh(range(len(top_caracteristicas)), top_caracteristicas['importancia'])
plt.yticks(range(len(top_caracteristicas)), top_caracteristicas['caracteristica'])
plt.xlabel('Importancia de Características')
plt.title('Top 10 Características Más Importantes - Modelo LightGBM')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# =============================================================================
# ANÁLISIS DE IMPACTO EMPRESARIAL
# =============================================================================

# Segmentación de riesgo
probabilidades_riesgo = y_prob_optimizado
clientes_alto_riesgo = np.sum(probabilidades_riesgo > 0.7)
clientes_riesgo_medio = np.sum((probabilidades_riesgo > 0.4) & (probabilidades_riesgo <= 0.7))
clientes_bajo_riesgo = np.sum(probabilidades_riesgo <= 0.4)

print(f"\nResultados de Segmentación de Riesgo:")
print(f"Clientes alto riesgo (>70%): {clientes_alto_riesgo} ({clientes_alto_riesgo/len(probabilidades_riesgo)*100:.1f}%)")
print(f"Clientes riesgo medio (40-70%): {clientes_riesgo_medio} ({clientes_riesgo_medio/len(probabilidades_riesgo)*100:.1f}%)")
print(f"Clientes bajo riesgo (<40%): {clientes_bajo_riesgo} ({clientes_bajo_riesgo/len(probabilidades_riesgo)*100:.1f}%)")

# Simulación de impacto empresarial
total_clientes = len(df)
tasa_abandono_anual = df['Exited'].sum() / total_clientes
clientes_abandonan_anualmente = int(total_clientes * tasa_abandono_anual)
clientes_detectados = int(clientes_abandonan_anualmente * recall)

# Impacto financiero (valores ejemplo)
valor_promedio_cliente = 1200  # Valor anual por cliente
costo_retencion = 50          # Costo de campaña de retención
costo_adquisicion = 200       # Costo de adquirir nuevo cliente

ingresos_salvados = clientes_detectados * valor_promedio_cliente
costo_campana = clientes_alto_riesgo * costo_retencion
beneficio_neto = ingresos_salvados - costo_campana

print(f"\nSimulación de Impacto Empresarial:")
print(f"Total de clientes: {total_clientes:,}")
print(f"Tasa de abandono anual: {tasa_abandono_anual:.1%}")
print(f"Clientes que abandonan anualmente: {clientes_abandonan_anualmente}")
print(f"Clientes detectados por el modelo: {clientes_detectados} ({recall:.1%} tasa de detección)")
print(f"Ingresos salvados: {ingresos_salvados:,}€")
print(f"Costos de campaña: {costo_campana:,}€")
print(f"Beneficio neto: {beneficio_neto:,}€")
print(f"ROI: {(beneficio_neto/costo_campana)*100:.1f}%")

# =============================================================================
# PERSISTENCIA DEL MODELO
# =============================================================================

import joblib
from datetime import datetime

# Guardar el modelo entrenado
archivo_modelo = f"../models/modelo_abandono_bancario_{datetime.now().strftime('%Y%m%d')}.pkl"
joblib.dump(modelo_optimizado, archivo_modelo)

# Guardar el escalador
archivo_escalador = f"../models/escalador_caracteristicas_{datetime.now().strftime('%Y%m%d')}.pkl"
joblib.dump(escalador, archivo_escalador)

print("¡Modelo entrenado listo para usarse!")

print(f"\nModelo guardado exitosamente:")
print(f"Archivo modelo: {archivo_modelo}")
print(f"Archivo escalador: {archivo_escalador}")

# Metadatos del modelo
metadatos_modelo = {
    'tipo_modelo': 'LightGBM',
    'rendimiento': {
        'puntaje_auc': float(auc_optimizado),
        'precision': float(precision_final),
        'precision_score': float(precision),
        'recall': float(recall),
        'f1_score': float(f1)
    },
    'hiperparametros': busqueda_grilla.best_params_,
    'nombres_caracteristicas': nombres_caracteristicas,
    'fecha_entrenamiento': datetime.now().isoformat(),
    'impacto_empresarial': {
        'tasa_deteccion': float(recall),
        'clientes_alto_riesgo': int(clientes_alto_riesgo),
        'beneficio_anual_estimado': int(beneficio_neto)
    }
}

print(f"\nProyecto completado exitosamente!")
print(f"AUC final del modelo: {auc_optimizado:.4f}")
print(f"Modelo listo para implementación en producción.")

: 